# Use local data to configure JBrowse views in Jupyter Notebooks

This Jupyter notebook will show you how to add tracks using your local data. We are able to leverage Jupyter's server to serve local files that exist within the notebook's file tree, but we also provide a separate dev server for additional support in local environments.

## Overview:
- Create an hg19 config
- Add Tracks utilizing local data
    - Add a feature track using local data from within the Jupyter File Tree
    - Add a feature track by spinning JBrowse Jupyter's dev server
- launch a linear genome view with a default session
  

## Resources
* [Here are the docs on how Jupyter handles urls and  file navigation with /tree.](https://jupyterlab.readthedocs.io/en/stable/user/urls.html)
* [Additional information on the Jupyter Server](https://jupyter-server.readthedocs.io/en/latest/users/configuration.html)
* [Additional information on the Jupyter Notebook Server](https://jupyter-notebook.readthedocs.io/en/stable/notebook.html#starting-the-notebook-server)

In [1]:
!pip install git+https://github.com/GMOD/jbrowse-jupyter.git@sc_paper_modifications
!pip install requests
!pip install jupyter_server_proxy
!pip install jupyter_dash

  Cloning https://github.com/GMOD/jbrowse-jupyter.git (to revision sc_paper_modifications) to /tmp/pip-req-build-p1wumnei
  Running command git clone --filter=blob:none --quiet https://github.com/GMOD/jbrowse-jupyter.git /tmp/pip-req-build-p1wumnei
  Running command git checkout -b sc_paper_modifications --track origin/sc_paper_modifications
  Switched to a new branch 'sc_paper_modifications'
  Branch 'sc_paper_modifications' set up to track remote branch 'sc_paper_modifications' from 'origin'.
  Resolved https://github.com/GMOD/jbrowse-jupyter.git to commit d8251f2613db7831dc4898f5f0633fd1b8939533
  Preparing metadata (setup.py) ... done


In [2]:
from jbrowse_jupyter import launch, create, serve
from IPython.display import FileLink, FileLinks
#from jupyter_dash import JupyterDash
import jupyter_server_proxy
import requests

In [3]:
#suggested by https://github.com/plotly/jupyter-dash/issues/28
from jupyter_dash.comms import _send_jupyter_config_comm_request
_send_jupyter_config_comm_request()

In [4]:
from jupyter_dash import JupyterDash

In [6]:
# Run this only if you are running the notebook in Jupyter not VSCode
JupyterDash.infer_jupyter_proxy_config() 

In [7]:
## Create an Hg19 JBrowse config
hg19 = create('LGV', genome='hg19')

## Adding Tracks from local data

### Using the Jupyter server 

This section show you how to use local files with jbrowse jupyter when your files exist within your jupyterlab/jupyternotebook file system. With the help of Jupyter Urls and file navigation with /tree, we are able to access the data to pass to JBrowse Jupyter's API.

**1.** Download our example gff files to add a track to our hg19 config.

> **_NOTE:_** we are only downloading these files for the tutorial. If you have  local files that you may want to use and they are already in your jupyter notebook directory, you will be able to go straight to step 3.

In [ ]:
file1 = 'https://s3.amazonaws.com/jbrowse.org/genomes/hg19/ncbi_refseq/GRCh37_latest_genomic.sort.gff.gz'
file2 = 'https://s3.amazonaws.com/jbrowse.org/genomes/hg19/ncbi_refseq/GRCh37_latest_genomic.sort.gff.gz.tbi'
response1 = requests.get(file1)
response2 = requests.get(file2)
open("GRCh37_latest_genomic.sort.gff.gz", "wb").write(response1.content)
open("GRCh37_latest_genomic.sort.gff.gz.tbi", "wb").write(response2.content)


**2.** Verify that local files exist within your notebook file system.

* Now that we have these new files in your file tree, we will be able to format urls to pass on to our components.

* Navigate to *http://localhost:8888/tree* or http://<your-host:your-port>/tree to verify that *GRCh37_latest_genomic.sort.gff.gz* and *GRCh37_latest_genomic.sort.gff.gz.tbi* exist within the file tree.

> **_NOTE_:** Your port may be different, so make sure that you use the same port from your jupyter notebook configuration. To ensure you have the correct one, you can look at the port in the url window where your notebook is running.

**3.** Format the Jupyter URLs of your local data to pass to your JBrowse view. 


* To format your urls: `http://localhost:8888/files/<your_file_name>`
* e.g 'http://localhost:8888/files/' + 'GRCh37_latest_genomic.sort.gff.gz' = 'http://localhost:8888/files/GRCh37_latest_genomic.sort.gff.gz'


> **_NOTE_:** You do not need to add tree or lab to this url.

In [8]:
# With the gff files that we just downloaded to our directory, we will add a track.
track_data = 'http://localhost:8888/files/GRCh37_latest_genomic.sort.gff.gz'
track_data_index = 'http://localhost:8888/files/GRCh37_latest_genomic.sort.gff.gz.tbi'
hg19.add_track(track_data, index=track_data_index, track_id='local_file_support_demo')
hg19.set_default_session(['local_file_support_demo'],False)

### Using the JBrowse Jupyter dev server

**1.** git clone our repo: [https://github.com/GMOD/jbrowse-jupyter](https://github.com/GMOD/jbrowse-jupyter). You can skip this step if you are already running this tutorial from within our repo.

**2.** From the root of our directory, run 
```$ python serve.py```

**3.** You can provide a specific PORT, HOST, and DIRECTORY from where you want to serve your data.
None of these are required. The PORT will default to 8080, the host will default to 'localhost', and the directory will default to your current working directory. You can move the files to exist within the repo or provide a path to your local data. 


Your terminal should look something like this, if no input was provided.
```
> python serve.py 
ENTER A PORT: 
ENTER A HOST: 
ENTER PATH TO YOUR LOCAL DATA: 

Warning, this is a development environment.
This is not Recommended for production.
Server is now running at 
 "http://localhost:8080"
```
**4.** Navigate to the url and verify that your local data is there. Now you should be able to provide urls to your local data to the JBrowse Jupyter config. Below you can find an example of our dev server serving files on port 8080 on local host.


You can also run the server by using the serve() method from our package. Checkout our docs for more information regarding our local dev server:
https://github.com/GMOD/jbrowse-jupyter/blob/main/README.md


> **_NOTE_:** You can skip the following two cells if you did not run the local dev server provided by JBrowse Jupyter. 


In [ ]:
track_data2 = 'http://localhost:8080/GRCh37_latest_genomic.sort.gff.gz'
track_data_index2 = 'http://localhost:8080/GRCh37_latest_genomic.sort.gff.gz.tbi'
hg19.add_track(track_data2, index=track_data_index2, track_id='dev_server_demo')

In [9]:
hg19.set_default_session(['local_file_support_demo'],False)

In [10]:
config = hg19.get_config()

In [11]:
launch(config, port=3000)

## FAQ & Troubleshooting
Issues with the JupyterDash proxy?

* This tutorial works in both jupyter notebook and jupyter lab. If you have issues with the proxy and you are working in jupyter lab, make sure that you have the `jupyterlab-dash` extension installed. If you get an error when installing it or after installing it, exit jupyter lab then run `$ jupyter lab build` and run jupyter lab again.

Issues with the PORT?

* Make sure that the PORT that your notebook is running is different from the one that you use to launch the JBrowse view and or the JBrowse Dev Server. To change the PORT in JBrowse Jupyter, simply pass a new port number to the launch() method or pass a different port to the dev server. 

Issues Jupyter Dash?

* Our package has dependencies on JupyterDash and Dash, please make sure that you are using jupyter lab version 2.0 and greater. This tutorial is currently working with the lastest version of jupyter lab 3.4.6.


## Contact Us
For other issues, please open a ticket on our repo or contact our gitter channel for more support

Report a bug or request a feature at https://github.com/GMOD/jbrowse-jupyter/issues
Join our developers chat at https://gitter.im/GMOD/jbrowse2
Send an email to our mailing list at gmod-ajax@lists.sourceforge.net

